# Generate and Submit Job Scripts
This note is used to generate job scripts and submit generated jobs.

## Generate Job Scripts
### Environments

In [1]:
# Environment
# Box2D
Box2D_continuous_env_set = ['BipedalWalker-v2', 'BipedalWalkerHardcore-v2', 'LunarLanderContinuous-v2']
Box2D_discrete_env_set = ['LunarLander-v2']
# Classic control
Classic_continuous_env_set = ['Acrobot-v1', 'CartPole-v1', 'MountainCar-v0']
Classic_discrete_env_set = ['MountainCarContinuous-v0', 'Pendulum-v0']
# TODO: MuJoCo
MuJoCo_continuous_env_set = []
MuJoCo_discrete_env_set = []


### Different parameters are investigated:
* random seed
* exploration strategy
* target network update method
* double DDPG

In [5]:
import os

# Constant
JOB_GPU = 0
JOB_CPU = 1
JOB_MEMORY = '8000M'
JOB_TIME = '27:00:00' # time formats "hours:minutes:seconds"
PROCESS_SCRIPT = './ddpg_discrete_action.py'
SUMMARY_DIR = '../Double_DDPG_Results_no_monitor' # directory save job output
JOB_SCRIPT_DIR = './job_scripts'
USE_GYM_MONITOR = False

# Investigated Parameters
action_space_types = ['continuous', 'discrete']
environments = {'continuous':['BipedalWalker-v2', 'BipedalWalkerHardcore-v2', 'LunarLanderContinuous-v2',
                              'MountainCarContinuous-v0', 'Pendulum-v0' ],
                'discrete':['LunarLander-v2',
                            'Acrobot-v1', 'CartPole-v1', 'MountainCar-v0']}
# Continuous tasks:
# 1. BipedalWalker-v2        : 2019, Feb. 14, 23:30 Cedar (done)
# 2. BipedalWalkerHardcore-v2: 2019, Feb. 15, 00:50 Cedar (partially done)
# 3. LunarLanderContinuous-v2: 2019, Feb. 17, 00:55 
# 4. MountainCarContinuous-v0: 2019, Feb. 17, 01:35 
# 5. Pendulum-v0             : 2019, Feb. 17, 20:05 Graham(submitted)

# Discrete tasks:
# 1. LunarLander-v2          : 2019, Feb. 17, 01:26
# 2. Acrobot-v1              : 2019, Feb. 17, 01:05 
# 3. CartPole-v1             : 2019, Feb. 17, 01:15
# 4. MountainCar-v0          : 2019, Feb. 17, 01:21

# environments = {'continuous':[],
#                 'discrete':['Pendulum-v0']}

ddpg_methods = ['ddpg', 'doule_ddpg']
target_net_update_method = ['softcopy', 'hardcopy']
exploration_strategies = ['action_noise', 'epsilon_greedy']
random_seeds = [1, 2, 3, 4, 5]
run_num = 10

job_count = 0
# Create Job Scripts:
# 1. Action space type
for action_space_type in action_space_types:
    if action_space_type == 'continuous':
        action_space_type_flag = '--continuous-act-space-flag'
    elif action_space_type == 'discrete':
        action_space_type_flag = ''
    else:
        print('Wrong action space type!')
    # 2. Environments
    for env in environments[action_space_type]:
        # 3. DDPG method: ddpg or double-ddpg
        for ddpg_name in ddpg_methods:
            if ddpg_name == 'ddpg':
                ddpg_methods_flag = '--double-ddpg-flag'
            elif ddpg_name == 'doule_ddpg':
                ddpg_methods_flag = ''
            else:
                print('Wrong ddpg method!')
            # 4. Target net update methods
            for target_net_update in target_net_update_method:
                if target_net_update == 'softcopy':
                    target_hard_copy_flag = ''
                elif target_net_update == 'hardcopy':
                    target_hard_copy_flag = '--target-hard-copy-flag'
                else:
                    print('Wrong target_net_update!')
                # 5. Exploration strategies    
                for exploration in exploration_strategies:
                    # 6. Seeds
                    for seed in random_seeds:
                        # 7. Runs
                        for run in range(run_num):
                            # Job Name Example: continuous_env_ddpg_softcopy_action_noise_seed1_run1.sh
                            job_name = '{0}_{1}_{2}_{3}_{4}_seed{5}_run{6}'.format(action_space_type,
                                                                                   env, 
                                                                                   ddpg_name,
                                                                                   target_net_update,
                                                                                   exploration,
                                                                                   seed,
                                                                                   run)
                            job_script_name = '{}.sh'.format(job_name)
                            # Summary Dir Example: ../Double_DDPG_Results/continuous/env/ddpg_softcopy_action_noise_seed1_run1
                            summary_dir = os.path.join(SUMMARY_DIR,
                                                       action_space_type, 
                                                       env,
                                                       '{0}_{1}_{2}_seed{3}_run{4}'.format(ddpg_name,
                                                                                           target_net_update,
                                                                                           exploration,
                                                                                           seed,
                                                                                           run))
                            # Job Script Example: ./job_scripts/continuous_env_ddpg_softcopy_action_noise_seed1_run1.sh
                            job_script_dir = os.path.join(JOB_SCRIPT_DIR, env)
                            if not os.path.exists(job_script_dir):
                                os.makedirs(job_script_dir)
                            with open(os.path.join(job_script_dir, job_script_name), 'w') as job_file:
                                # Write job script
                                job_file.write('#!/bin/bash\n')
                                job_file.write('#SBATCH --account=def-dkulic\n')
                                job_file.write('#SBATCH --gres=gpu:{}       # request GPU generic resource\n'.format(JOB_GPU))
                                job_file.write('#SBATCH --cpus-per-task={}  #Maximum of CPU cores per GPU request: 6 on Cedar, 16 on Graham.\n'.format(JOB_CPU))
                                job_file.write('#SBATCH --mem={}            # memory per node\n'.format(JOB_MEMORY))
                                job_file.write('#SBATCH --time={}           # time (DD-HH:MM)\n'.format(JOB_TIME))
#                                 job_file.write('#SBATCH --output={}/{}_%N-%j.out  # %N for node name, %j for jobID\n'.format(summary_dir, job_name))
                                job_file.write('#SBATCH --output={}/{}_%N-%j.out  # %N for node name, %j for jobID\n'.format('../Double_DDPG_Job_output', job_name))
                                job_file.write('\n')
                                job_file.write('module load cuda cudnn\n')
                                job_file.write('source ~/tf_gpu/bin/activate\n')
                                job_file.write('\n')
                                if USE_GYM_MONITOR:
                                    use_gym_monitor_flag = '--use-gym-monitor-flag'
                                else:
                                    use_gym_monitor_flag = ''
                                
                                # Example: python './ddpg_discrete_action.py' --env BipedalWalker-v2 --continuous-act-space-flag --random-seed 1 --exploration-strategy action_noise --use-gym-monitor-flag --summary-dir ../Double_DDPG_Results/continuous/env/ddpg_softcopy_action_noise_seed1_run1
                                main_command = "python {0} --env {1} --random-seed {2} --exploration-strategy {3} --summary-dir {4} {5} {6} {7} {8}\n".format(\
                                PROCESS_SCRIPT, 
                                env, 
                                seed, 
                                exploration, 
                                summary_dir, 
                                action_space_type_flag, 
                                ddpg_methods_flag, 
                                target_hard_copy_flag, 
                                use_gym_monitor_flag)
                                
                                if USE_GYM_MONITOR:
                                    job_file.write("{0} {1}\n".format("xvfb-run -a ", 
                                                                    main_command))
                                else:
                                    job_file.write("{0}\n".format(main_command))
                                job_count += 1
                                print('Job {}: {}'.format(job_count, job_script_name))
                                

Job 1: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run0.sh
Job 2: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run1.sh
Job 3: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run2.sh
Job 4: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run3.sh
Job 5: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run4.sh
Job 6: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run5.sh
Job 7: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run6.sh
Job 8: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run7.sh
Job 9: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run8.sh
Job 10: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed1_run9.sh
Job 11: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed2_run0.sh
Job 12: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed2_run1.sh
Job 13: continuous_BipedalWalker-v2_ddpg_softcopy_action_noise_seed2_run2.sh
Job 14: 

Job 116: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed2_run5.sh
Job 117: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed2_run6.sh
Job 118: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed2_run7.sh
Job 119: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed2_run8.sh
Job 120: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed2_run9.sh
Job 121: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run0.sh
Job 122: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run1.sh
Job 123: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run2.sh
Job 124: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run3.sh
Job 125: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run4.sh
Job 126: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run5.sh
Job 127: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_seed3_run6.sh
Job 128: continuous_BipedalWalker-v2_ddpg_hardcopy_action_noise_

Job 269: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed2_run8.sh
Job 270: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed2_run9.sh
Job 271: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run0.sh
Job 272: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run1.sh
Job 273: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run2.sh
Job 274: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run3.sh
Job 275: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run4.sh
Job 276: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run5.sh
Job 277: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run6.sh
Job 278: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run7.sh
Job 279: continuous_BipedalWalker-v2_doule_ddpg_softcopy_epsilon_greedy_seed3_run8.sh
Job 280: continuous_BipedalWalker-v2_doule_ddpg_softco

Job 381: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run0.sh
Job 382: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run1.sh
Job 383: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run2.sh
Job 384: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run3.sh
Job 385: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run4.sh
Job 386: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run5.sh
Job 387: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run6.sh
Job 388: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run7.sh
Job 389: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run8.sh
Job 390: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed4_run9.sh
Job 391: continuous_BipedalWalker-v2_doule_ddpg_hardcopy_epsilon_greedy_seed5_run0.sh
Job 392: continuous_BipedalWalker-v2_doule_ddpg_hardco

Job 490: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed4_run9.sh
Job 491: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run0.sh
Job 492: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run1.sh
Job 493: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run2.sh
Job 494: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run3.sh
Job 495: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run4.sh
Job 496: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run5.sh
Job 497: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run6.sh
Job 498: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run7.sh
Job 499: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run8.sh
Job 500: continuous_BipedalWalkerHardcore-v2_ddpg_softcopy_epsilon_greedy_seed5_run9.sh
Job 501: continuous_BipedalWalke

...

Job 3022: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run1.sh
Job 3023: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run2.sh
Job 3024: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run3.sh
Job 3025: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run4.sh
Job 3026: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run5.sh
Job 3027: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run6.sh
Job 3028: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run7.sh
Job 3029: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run8.sh
Job 3030: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed3_run9.sh
Job 3031: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed4_run0.sh
Job 3032: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed4_run1.sh
Job 3033: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_seed4_run2.sh
Job 3034: discrete_CartPole-v1_doule_ddpg_softcopy_action_noise_

Job 3128: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run7.sh
Job 3129: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run8.sh
Job 3130: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed3_run9.sh
Job 3131: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run0.sh
Job 3132: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run1.sh
Job 3133: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run2.sh
Job 3134: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run3.sh
Job 3135: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run4.sh
Job 3136: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run5.sh
Job 3137: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run6.sh
Job 3138: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run7.sh
Job 3139: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_seed4_run8.sh
Job 3140: discrete_CartPole-v1_doule_ddpg_hardcopy_action_noise_

Job 3250: discrete_MountainCar-v0_ddpg_softcopy_action_noise_seed5_run9.sh
Job 3251: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run0.sh
Job 3252: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run1.sh
Job 3253: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run2.sh
Job 3254: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run3.sh
Job 3255: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run4.sh
Job 3256: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run5.sh
Job 3257: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run6.sh
Job 3258: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run7.sh
Job 3259: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run8.sh
Job 3260: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed1_run9.sh
Job 3261: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run0.sh
Job 3262: discrete_MountainCar-v0_ddpg_softcopy_epsilon_greedy_seed2_run1.sh
J

Job 3374: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed3_run3.sh
Job 3375: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed3_run4.sh
Job 3376: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed3_run5.sh
Job 3377: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed3_run6.sh
Job 3378: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed3_run7.sh
Job 3379: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed3_run8.sh
Job 3380: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed3_run9.sh
Job 3381: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run0.sh
Job 3382: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run1.sh
Job 3383: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run2.sh
Job 3384: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run3.sh
Job 3385: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run4.sh
Job 3386: discrete_MountainCar-v0_ddpg_hardcopy_epsilon_greedy_seed4_run5.sh

Job 3499: discrete_MountainCar-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run8.sh
Job 3500: discrete_MountainCar-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run9.sh
Job 3501: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run0.sh
Job 3502: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run1.sh
Job 3503: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run2.sh
Job 3504: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run3.sh
Job 3505: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run4.sh
Job 3506: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run5.sh
Job 3507: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run6.sh
Job 3508: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run7.sh
Job 3509: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run8.sh
Job 3510: discrete_MountainCar-v0_doule_ddpg_hardcopy_action_noise_seed1_run9.sh
Job 3511: discrete_Mount

## Submit Job Scripts

In [4]:
import os
JOB_SCRIPT_DIR = './job_scripts'
job_scripts_dir = os.path.join(JOB_SCRIPT_DIR, 'Pendulum-v0')
job_scripts_list = os.listdir(job_scripts_dir)

for job in job_scripts_list:
    print('Submitting job: {}'.format(job))
    code = os.system('sbatch {}'.format(os.path.join(job_scripts_dir, job)))
    print('{}: {}'.format(job, code))
    

Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run1.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run1.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed2_run2.sh
continuous_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed2_run2.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed5_run4.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed5_run4.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run2.sh
discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run2.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed1_run3.sh
continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed1_run3.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed2_run1.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed2_run1.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_softcopy_epsil

discrete_Pendulum-v0_doule_ddpg_softcopy_action_noise_seed2_run4.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run3.sh
continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run3.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed5_run5.sh
continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed5_run5.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed2_run2.sh
discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed2_run2.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed4_run2.sh
discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed4_run2.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run8.sh
continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run8.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run9.sh
continuous_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run9.sh: 0
Submitting job: con

continuous_Pendulum-v0_ddpg_hardcopy_action_noise_seed4_run8.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run9.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run9.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed5_run9.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed5_run9.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run2.sh
discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run2.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed2_run9.sh
continuous_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed2_run9.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed1_run3.sh
discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed1_run3.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run4.sh
discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run4.sh: 0
Submitting job: continu

continuous_Pendulum-v0_ddpg_softcopy_action_noise_seed1_run1.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed4_run3.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed4_run3.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed2_run2.sh
discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed2_run2.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed4_run8.sh
continuous_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed4_run8.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run8.sh
continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run8.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run7.sh
discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run7.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run5.sh
continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run5.sh: 0
Submitt

discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run4.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run8.sh
discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run8.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_hardcopy_action_noise_seed1_run0.sh
continuous_Pendulum-v0_ddpg_hardcopy_action_noise_seed1_run0.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed2_run7.sh
discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed2_run7.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run6.sh
discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run6.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed1_run8.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed1_run8.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed3_run9.sh
discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed3_run9.sh: 0
Submitting job: discrete_Pendulum

...

discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed3_run2.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed3_run0.sh
discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed3_run0.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed2_run7.sh
continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed2_run7.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed2_run7.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed2_run7.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed1_run1.sh
discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed1_run1.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run5.sh
discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed5_run5.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed1_run6.sh
continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed1_run6.sh: 0
Submitting jo

discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed2_run2.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed3_run5.sh
discrete_Pendulum-v0_ddpg_hardcopy_action_noise_seed3_run5.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_hardcopy_action_noise_seed3_run4.sh
continuous_Pendulum-v0_ddpg_hardcopy_action_noise_seed3_run4.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed3_run4.sh
discrete_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed3_run4.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed2_run0.sh
continuous_Pendulum-v0_doule_ddpg_softcopy_epsilon_greedy_seed2_run0.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_hardcopy_action_noise_seed2_run9.sh
continuous_Pendulum-v0_ddpg_hardcopy_action_noise_seed2_run9.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed3_run9.sh
continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed3_run9.sh: 0
Submitting job: d

discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run5.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_softcopy_action_noise_seed4_run6.sh
discrete_Pendulum-v0_doule_ddpg_softcopy_action_noise_seed4_run6.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_softcopy_action_noise_seed5_run2.sh
continuous_Pendulum-v0_ddpg_softcopy_action_noise_seed5_run2.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed5_run7.sh
continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed5_run7.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed1_run9.sh
continuous_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed1_run9.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed2_run0.sh
discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed2_run0.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run6.sh
discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run6.sh: 0
Submitting job: discrete_Pe

discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed3_run7.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed5_run7.sh
discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed5_run7.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed2_run5.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed2_run5.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed5_run9.sh
discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed5_run9.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run6.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed3_run6.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_softcopy_action_noise_seed4_run0.sh
discrete_Pendulum-v0_doule_ddpg_softcopy_action_noise_seed4_run0.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed5_run0.sh
discrete_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed5_run0.sh: 0
Submitting job: continuous_Pend

continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed1_run5.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed3_run0.sh
continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed3_run0.sh: 0
Submitting job: discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed5_run1.sh
discrete_Pendulum-v0_doule_ddpg_hardcopy_epsilon_greedy_seed5_run1.sh: 0
Submitting job: continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed2_run1.sh
continuous_Pendulum-v0_ddpg_softcopy_epsilon_greedy_seed2_run1.sh: 0
Submitting job: continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed5_run2.sh
continuous_Pendulum-v0_doule_ddpg_hardcopy_action_noise_seed5_run2.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run3.sh
discrete_Pendulum-v0_ddpg_softcopy_action_noise_seed4_run3.sh: 0
Submitting job: discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed4_run8.sh
discrete_Pendulum-v0_ddpg_hardcopy_epsilon_greedy_seed4_run8.sh: 0
Submitting job: discr